In [ ]:
import numpy as np
import tensorflow as tf
%tensorflow_version 2.x
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
!pip install transformers
import torch
from torch import nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
path = "https://raw.githubusercontent.com/MU5HF1QUR/Thesis/main/Bengali%20hate%20speech%20.csv"
import pandas as pd

df = pd.read_csv(path)
df=df.dropna().reset_index(drop=True)


#print(df.info())
display(df)

,sentence,hate,category
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports
...,...,...,...
29995,আমার মনে হচ্ছে মেনে নেয়া উচিত,0,"Meme, TikTok and others"
29996,আমি ধন্যবাদ জানাই আইনপসাসনকে,0,"Meme, TikTok and others"
29997,কাসমির কাসমিরই নিজশ্যই সাদিন হওয়ার দরকার,0,"Meme, TikTok and others"
29998,কলমি পিলিজ আপু মনি অনেক কিওট লাগছে,0,"Meme, TikTok and others"


In [ ]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations

                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)

    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
def stopwordRemoval(text):
    x=str(text)
    l=x.split()
    stop = stopwords.words('bengali')

    stm=[elem for elem in l if elem not in stop]

    out=' '.join(stm)

    return str(out)

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰⚽️✌�￰৷￰'''

    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out


def Stemming(text):

    x=str(text)
    l=x.split()

    stmr = stemmer.BanglaStemmer()
    stm = stmr.stem(l)

    out=' '.join(stm)

    return str(out)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install bangla-stemmer

In [ ]:
from bangla_stemmer.stemmer import stemmer

In [ ]:
df['sentence'] = df.sentence.apply(lambda x: preprocessing(str(x)))

df['sentence'] = df.sentence.apply(lambda x: Stemming(str(x)))

df['sentence'] = df.sentence.apply(lambda x: stopwordRemoval(str(x)))

Streaming output truncated to the last 5000 lines.
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules

# Model_Run

In [ ]:
texts = df['sentence']
labels = df['hate']
print(texts)
print(labels)

0                                 যত্তসব পাপন শালা ফাজলামী
1                                পাপন শালা রে রিমান্ডে দরক
2        জিল্লুর রহমান স্যার ছাল বড় জারজ এক দেশের মানুষ...
3                              শালা লুচ্চা দেখ পাঠা মত যায়
4             তুই শালা গাজা খাইছচতুর মার হেডায় খেলবে সাকিব
                               ...                        
29995                                      আম হচ্ মান নেয়া
29996                                   ধন্যবাদ আইনপসাসনকে
29997                   কাসমির কাসমির নিজশ্য সাদিন হওয় দরক
29998                          কলমি পিলিজ আপু মনি কিওট লাগ
29999                                  পাকিস্তান সাথে জড়িত
Name: sentence, Length: 30000, dtype: object
0        1
1        1
2        1
3        1
4        1
        ..
29995    0
29996    0
29997    0
29998    0
29999    0
Name: hate, Length: 30000, dtype: int64


## RNN

In [ ]:
import numpy as np
import tensorflow as tf
%tensorflow_version 2.x
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
!pip install transformers
import torch
from torch import nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 'df' is the DataFrame
data_binary = df[df['hate'] == 1]
data_multilevel = df[(df['hate'] == 1) & ~df['category'].isnull()]

# Tokenize and pad the input sequences for binary classification
tokenizer_binary = Tokenizer()
tokenizer_binary.fit_on_texts(df['sentence'])
sequences_binary = tokenizer_binary.texts_to_sequences(df['sentence'])
vocab_size_binary = len(tokenizer_binary.word_index) + 1
max_len_binary = max([len(seq) for seq in sequences_binary])
padded_sequences_binary = pad_sequences(sequences_binary, maxlen=max_len_binary)

# Convert labels to NumPy array for binary classification
labels_binary = df['hate'].values

# Split the dataset into training and testing
train_sequences_binary, test_sequences_binary, labels_binary_train, labels_binary_test = train_test_split(
    padded_sequences_binary, labels_binary, test_size=0.2, random_state=42
)

# RNN model for binary classification
model_binary = Sequential()
model_binary.add(Embedding(vocab_size_binary, 100, input_length=max_len_binary))
model_binary.add(LSTM(128))
model_binary.add(Dense(1, activation='sigmoid'))
# Compile the model for binary classification
model_binary.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model for binary classification
model_binary.fit(train_sequences_binary, labels_binary_train, epochs=10, batch_size=16)

# Evaluate the model for binary classification
predictions_binary = (model_binary.predict(test_sequences_binary) > 0.5).astype('int32')
accuracy_binary = accuracy_score(labels_binary_test, predictions_binary)
f1_binary = f1_score(labels_binary_test, predictions_binary)
report_binary = classification_report(labels_binary_test, predictions_binary, target_names=['class 0', 'class 1'])
print('Binary Classification Report:\n', report_binary)

# Convert category labels to numerical format for multilevel classification
label_encoder = LabelEncoder()
labels_multilevel = label_encoder.fit_transform(data_multilevel['category'])

# Tokenize and pad the input sequences for multilevel classification
tokenizer_multilevel = Tokenizer()
tokenizer_multilevel.fit_on_texts(data_multilevel['sentence'])
sequences_multilevel = tokenizer_multilevel.texts_to_sequences(data_multilevel['sentence'])
vocab_size_multilevel = len(tokenizer_multilevel.word_index) + 1
max_len_multilevel = max([len(seq) for seq in sequences_multilevel])
padded_sequences_multilevel = pad_sequences(sequences_multilevel, maxlen=max_len_multilevel)

# Split the dataset into training and testing
train_sequences_multilevel, test_sequences_multilevel, labels_multilevel_train, labels_multilevel_test = train_test_split(
    padded_sequences_multilevel, labels_multilevel, test_size=0.2, random_state=42
)

# RNN model for multilevel classification
model_multilevel = Sequential()
model_multilevel.add(Embedding(vocab_size_multilevel, 100, input_length=max_len_multilevel))
model_multilevel.add(LSTM(128))
model_multilevel.add(Dense(len(data_multilevel['category'].unique()), activation='softmax'))
# Compile the model for multilevel classification
model_multilevel.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model for multilevel classification
model_multilevel.fit(train_sequences_multilevel, labels_multilevel_train, epochs=10, batch_size=16)

# Evaluate the model for multilevel classification
predictions_multilevel = np.argmax(model_multilevel.predict(test_sequences_multilevel), axis=1)
accuracy_multilevel = accuracy_score(labels_multilevel_test, predictions_multilevel)
f1_multilevel = f1_score(labels_multilevel_test, predictions_multilevel, average='weighted')
report_multilevel = classification_report(labels_multilevel_test, predictions_multilevel, target_names=label_encoder.classes_)
print('Multilevel Classification Report:\n', report_multilevel)
print()

# Print Accuracy and F1 Score for Binary Classification
print("\nBinary Classification Metrics:")
print(f"Accuracy: {accuracy_binary:.4f}")
print(f"F1 Score: {f1_binary:.4f}")

# Print Accuracy and F1 Score for Multilevel Classification
print("\nMultilevel Classification Metrics:")
print(f"Accuracy: {accuracy_multilevel:.4f}")
print(f"F1 Score: {f1_multilevel:.4f}")



Epoch 1/10
1500/1500 [==============================] - 83s 51ms/step - loss: 0.3554 - accuracy: 0.8519
Epoch 2/10
1500/1500 [==============================] - 29s 19ms/step - loss: 0.1960 - accuracy: 0.9233
Epoch 3/10
1500/1500 [==============================] - 29s 19ms/step - loss: 0.1186 - accuracy: 0.9568
Epoch 4/10
1500/1500 [==============================] - 26s 18ms/step - loss: 0.0748 - accuracy: 0.9732
Epoch 5/10
1500/1500 [==============================] - 26s 17ms/step - loss: 0.0500 - accuracy: 0.9821
Epoch 6/10
1500/1500 [==============================] - 25s 17ms/step - loss: 0.0376 - accuracy: 0.9867
Epoch 7/10
1500/1500 [==============================] - 25s 17ms/step - loss: 0.0283 - accuracy: 0.9898
Epoch 8/10
1500/1500 [==============================] - 25s 17ms/step - loss: 0.0213 - accuracy: 0.9922
Epoch 9/10
1500/1500 [==============================] - 25s 16ms/step - loss: 0.0191 - accuracy: 0.9928
Epoch 10/10
188/188 [==============================] - 2s 6ms/st

## Multinomial NB, Logistic Regression, DecisionTreeClassifier

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import warnings

# Suppress Convergence Warnings
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')


X = df.sentence
Y = df.hate.values

# TF-IDF Vectorization
tfidf_v = TfidfVectorizer(min_df=5, ngram_range=(1, 2))
X = tfidf_v.fit_transform(X).toarray()

validation_split = 0.2

# Shuffle and split the data
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
Y = Y[indices]

num_validation_samples = int(validation_split * X.shape[0])

X_train_binary = X[:-num_validation_samples]
y_train_binary = Y[:-num_validation_samples]
X_test_binary = X[-num_validation_samples:]
y_test_binary = Y[-num_validation_samples:]

# Binary Classification Models
models_binary = {
    "Multinomial NB": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),  # Increase max_iter to avoid convergence warning
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

# Function for fitting and scoring binary classification models
def fit_and_score_binary(models, X_train, X_test, y_train, y_test):
    model_scores = {}

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        model_scores[name] = {'Accuracy': accuracy, 'F1 Score': f1}
    return model_scores

# Evaluate binary classification models
models_scores_binary = fit_and_score_binary(models_binary, X_train_binary, X_test_binary, y_train_binary, y_test_binary)
model_compare_binary = pd.DataFrame(models_scores_binary).T

print("Binary Classification Results:")
print(model_compare_binary)


df_hate = df[df['hate'] == 1]

# Split the data into training and testing sets for multilevel classification
X_hate = tfidf_v.transform(df_hate['sentence']).toarray()
y_hate = df_hate['category']
X_train_hate, X_test_hate, y_train_hate, y_test_hate = train_test_split(X_hate, y_hate, test_size=validation_split, random_state=47)

# Convert category labels to numerical values using LabelEncoder
le_hate = LabelEncoder()
y_train_hate_encoded = le_hate.fit_transform(y_train_hate)
y_test_hate_encoded = le_hate.transform(y_test_hate)

# Multilevel Classification Models
models_hate = {
    "Multinomial NB": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),  # Increase max_iter to avoid convergence warning
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

# Function for multilevel classification models
def fit_and_score_hate(models, X_train, X_test, y_train, y_test):
    model_scores = {}

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multiclass classification

        model_scores[name] = {'Accuracy': accuracy, 'F1 Score': f1}
    return model_scores


models_scores_hate = fit_and_score_hate(models_hate, X_train_hate, X_test_hate, y_train_hate_encoded, y_test_hate_encoded)
model_compare_hate = pd.DataFrame(models_scores_hate).T

print("\nMultilevel Classification Results on Hate Speech Instances:")
print(model_compare_hate)


Binary Classification Results:
                        Accuracy  F1 Score
Multinomial NB          0.735833  0.700955
Logistic Regression     0.745833  0.720270
DecisionTreeClassifier  0.697833  0.686275

Multilevel Classification Results on Hate Speech Instances:
                        Accuracy  F1 Score
Multinomial NB            0.4455  0.436336
Logistic Regression       0.4625  0.462944
DecisionTreeClassifier    0.3785  0.385521


## RFC

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder


train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


vectorizer = TfidfVectorizer()
train_features = vectorizer.fit_transform(train_texts)
test_features = vectorizer.transform(test_texts)


model = RandomForestClassifier()
model.fit(train_features, train_labels)


predictions = model.predict(test_features)


accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)
f1 = f1_score(test_labels, predictions)
print("F1 Score:", f1)




hate_speech_data = df[df['hate'] == 1]


train_texts, test_texts, train_category_labels, test_category_labels = train_test_split(
    hate_speech_data['sentence'], hate_speech_data['category'], test_size=0.2, random_state=42
)


vectorizer_category = TfidfVectorizer()
train_category_features = vectorizer_category.fit_transform(train_texts)
test_category_features = vectorizer_category.transform(test_texts)


le = LabelEncoder()


train_category_labels_encoded = le.fit_transform(train_category_labels)


model_category_classification = RandomForestClassifier()
model_category_classification.fit(train_category_features, train_category_labels_encoded)


predictions_category_classification = model_category_classification.predict(test_category_features)


predicted_categories = le.inverse_transform(predictions_category_classification)


accuracy_category_classification = accuracy_score(test_category_labels, predicted_categories)
f1_category_classification = f1_score(test_category_labels, predicted_categories, average='weighted')
print("Category Classification Accuracy on Hate Speech Instances:", accuracy_category_classification)
print("Category Classification F1 Score on Hate Speech Instances:", f1_category_classification)



Accuracy: 0.733
F1 Score: 0.47783572359843546
Category Classification Accuracy on Hate Speech Instances: 0.43
Category Classification F1 Score on Hate Speech Instances: 0.43960504807860523


In [ ]:

def predict_hate_and_category(sentence):

    sentence_vectorized = vectorizer.transform([sentence])


    is_hate_speech = model.predict(sentence_vectorized)[0]

    if is_hate_speech == 1:

        sentence_vectorized_category = vectorizer_category.transform([sentence])
        category_numerical = model_category_classification.predict(sentence_vectorized_category)[0]
        predicted_category = le.inverse_transform([category_numerical])[0]
        print("Hate Speech Prediction: Yes")
        print("Predicted Category:", predicted_category)
    else:
        print("Hate Speech Prediction: No")
        print("Predicted Category: Not Applicable")


input_sentence = "জিল্লুর রহমান এতো বড় জারজ হবে"
predict_hate_and_category(input_sentence)



Hate Speech Prediction: Yes
Predicted Category: religion


## CNN-LSTM Hybrid

In [ ]:
import pandas as pd
from pandas import read_excel
import numpy as np
import re
from re import sub
import multiprocessing

import os
from time import time
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Activation,Embedding,Flatten,Bidirectional,MaxPooling2D, Conv1D, MaxPooling1D
from keras.optimizers import SGD,Adam
from keras import regularizers
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import h5py
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
from tensorflow.keras.layers import Embedding, Conv1D, Bidirectional, LSTM, Dropout, Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 25000
embedding_dim = 300
max_length = 100
trunc_type='post'
oov_tok = "<OOV>"


train_binary, test_binary = train_test_split(df, random_state=69, test_size=0.2)


train_hate = df[df['hate'] == 1]
test_hate = df[df['hate'] == 1]

# Binary classification data
training_sentences_binary = train_binary['sentence'].values
testing_sentences_binary = test_binary['sentence'].values
train_labels_binary = keras.utils.to_categorical(train_binary['hate'].values)
test_labels_binary = keras.utils.to_categorical(test_binary['hate'].values)

# Multilevel classification data
training_sentences_hate = train_hate['sentence'].values
testing_sentences_hate = test_hate['sentence'].values

# converting 'category' column into numerical labels
le = LabelEncoder()
train_labels_hate = keras.utils.to_categorical(le.fit_transform(train_hate['category'].values))
test_labels_hate = keras.utils.to_categorical(le.transform(test_hate['category'].values))

num_classes = len(df['category'].unique())

#  for binary classification
tokenizer_binary = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer_binary.fit_on_texts(training_sentences_binary)
sequences_binary = tokenizer_binary.texts_to_sequences(training_sentences_binary)
padded_binary = pad_sequences(sequences_binary, maxlen=max_length, truncating=trunc_type)

test_sequences_binary = tokenizer_binary.texts_to_sequences(testing_sentences_binary)
testing_padded_binary = pad_sequences(test_sequences_binary, maxlen=max_length)

# for multilevel classification
tokenizer_hate = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer_hate.fit_on_texts(training_sentences_hate)
sequences_hate = tokenizer_hate.texts_to_sequences(training_sentences_hate)
padded_hate = pad_sequences(sequences_hate, maxlen=max_length, truncating=trunc_type)

test_sequences_hate = tokenizer_hate.texts_to_sequences(testing_sentences_hate)
testing_padded_hate = pad_sequences(test_sequences_hate, maxlen=max_length)

# Model for binary classification
model_binary = Sequential()
model_binary.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model_binary.add(Conv1D(200, kernel_size=3, activation="relu"))
model_binary.add(Bidirectional(LSTM(64, return_sequences=True)))
model_binary.add(Dropout(0.5))
model_binary.add(Bidirectional(LSTM(64)))
model_binary.add(Dense(50, activation='relu'))
model_binary.add(Dense(50, activation='relu'))
model_binary.add(Flatten())
model_binary.add(Dense(100, kernel_regularizer=regularizers.l2(0.01), activation="relu"))
model_binary.add(Dense(2, activation='softmax'))
adam = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
model_binary.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Model for multilevel classification
model_hate = Sequential()
model_hate.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model_hate.add(Conv1D(200, kernel_size=3, activation="relu"))
model_hate.add(Bidirectional(LSTM(64, return_sequences=True)))
model_hate.add(Dropout(0.5))
model_hate.add(Bidirectional(LSTM(64)))
model_hate.add(Dense(50, activation='relu'))
model_hate.add(Dense(50, activation='relu'))
model_hate.add(Flatten())
model_hate.add(Dense(100, kernel_regularizer=regularizers.l2(0.01), activation="relu"))
model_hate.add(Dense(num_classes, activation='softmax'))  # 'num_classes' = unique categories
adam = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
adam.build(model_hate.trainable_variables)
model_hate.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


history_binary = model_binary.fit(padded_binary, train_labels_binary, epochs=3, batch_size=256, validation_data=(testing_padded_binary, test_labels_binary), use_multiprocessing=True, workers=8)


history_hate = model_hate.fit(
    padded_hate,
    train_labels_hate,
    epochs=4,
    batch_size=256,
    validation_data=(testing_padded_hate, test_labels_hate),
    use_multiprocessing=True,
    workers=8
)

loss_and_metrics_binary = model_binary.evaluate(testing_padded_binary, test_labels_binary, batch_size=256)
print("Binary Classification Test Accuracy:", loss_and_metrics_binary[1])


loss_and_metrics_hate = model_hate.evaluate(testing_padded_hate, test_labels_hate, batch_size=256)
print("Multilevel Classification Test Accuracy:", loss_and_metrics_hate[1])


predictions_binary = model_binary.predict(testing_padded_binary)
predicted_labels_binary = np.argmax(predictions_binary, axis=1)
true_labels_binary = np.argmax(test_labels_binary, axis=1)


predictions_hate = model_hate.predict(testing_padded_hate)
predicted_labels_hate = np.argmax(predictions_hate, axis=1)
true_labels_hate = np.argmax(test_labels_hate, axis=1)


f1_binary = f1_score(true_labels_binary, predicted_labels_binary)
f1_hate = f1_score(true_labels_hate, predicted_labels_hate, average='weighted')  # Use 'weighted' for multiclass classification


print("Binary Classification F1 Score:", f1_binary)
print("Multilevel Classification F1 Score:", f1_hate)



Epoch 1/3
94/94 [==============================] - 23s 149ms/step - loss: 0.9938 - accuracy: 0.7469 - val_loss: 0.6785 - val_accuracy: 0.8648
Epoch 2/3
94/94 [==============================] - 11s 112ms/step - loss: 0.4962 - accuracy: 0.9031 - val_loss: 0.4769 - val_accuracy: 0.8792
Epoch 3/3
94/94 [==============================] - 9s 91ms/step - loss: 0.2921 - accuracy: 0.9393 - val_loss: 0.4194 - val_accuracy: 0.8807
Epoch 1/4
40/40 [==============================] - 16s 188ms/step - loss: 2.4915 - accuracy: 0.1987 - val_loss: 2.3958 - val_accuracy: 0.2000
Epoch 2/4
40/40 [==============================] - 6s 154ms/step - loss: 2.2886 - accuracy: 0.2471 - val_loss: 2.0485 - val_accuracy: 0.3278
Epoch 3/4
40/40 [==============================] - 5s 126ms/step - loss: 1.8851 - accuracy: 0.3577 - val_loss: 1.6224 - val_accuracy: 0.4828
Epoch 4/4
24/24 [==============================] - 0s 19ms/step - loss: 0.4194 - accuracy: 0.8807
Binary Classification Test Accuracy: 0.880666673183441

## mBERT

### binary classification

In [ ]:
# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, file_path):
        self.sentences, self.labels = self.load_data(file_path)

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        label = self.labels[index]

        return sentence, label

    def load_data(self, file_path):
      df = pd.read_csv(file_path)

      df = df.dropna().reset_index(drop=True)
      df['sentence'] = df.sentence.apply(preprocessing)
      df['sentence'] = df.sentence.apply(stopwordRemoval)
      df['sentence'] = df.sentence.apply(lambda x: Stemming(str(x)))
      df.info()
      sentences = df['sentence'].tolist()
      df['hate'] = df['hate'].astype(int)
      labels = df['hate'].tolist()

      return sentences, labels


# Load dataset
dataset_file_path = "https://raw.githubusercontent.com/MU5HF1QUR/Thesis/main/Bengali%20hate%20speech%20.csv"
dataset = CustomDataset(dataset_file_path)
sentences = dataset.sentences
labels = dataset.labels
print(len(dataset), type(dataset))
print(len(dataset.sentences), type(dataset.sentences))
print(len(dataset.labels), type(dataset.labels))


Streaming output truncated to the last 5000 lines.
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
a

In [ ]:

# Split the dataset into training and testing subsets
train_sentences, test_sentences, train_labels, test_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)


def tokenize_sentences(sentences, tokenizer):
    input_ids = []
    attention_masks = []

    for sentence in sentences:
        encoded_dict = tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=128,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)



train_input_ids, train_attention_masks = tokenize_sentences(train_sentences, tokenizer)


test_input_ids, test_attention_masks = tokenize_sentences(test_sentences, tokenizer)


batch_size = 16

train_data = TensorDataset(train_input_ids, train_attention_masks, torch.tensor(train_labels))
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)


test_data = TensorDataset(test_input_ids, test_attention_masks, torch.tensor(test_labels))
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()


test_accuracies = []
test_f1 = []

for epoch in range(3):
    model.train()

    for batch in train_dataloader:
        batch_input_ids = batch[0].to(device)
        batch_attention_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)

        logits = outputs.logits

        loss = loss_fn(logits, batch_labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

    model.eval()

    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        all_predicted_labels = []
        all_true_labels = []

        for batch in test_dataloader:
            batch_input_ids = batch[0].to(device)
            batch_attention_masks = batch[1].to(device)
            batch_labels = batch[2].to(device)

            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)
            logits = outputs.logits

            _, predicted_labels = torch.max(logits, dim=1)
            total_correct += (predicted_labels == batch_labels).sum().item()
            total_samples += batch_labels.size(0)

            all_predicted_labels.extend(predicted_labels.cpu().numpy())
            all_true_labels.extend(batch_labels.cpu().numpy())

        accuracy = total_correct / total_samples
        test_accuracies.append(accuracy)
        f1 = f1_score(all_true_labels, all_predicted_labels, average='macro')
        test_f1.append(f1)

        print(f"Epoch {epoch+1}: Test Accuracy = {accuracy * 100:.2f}%, F1 Score = {f1:.2f}")

# Calculate the average test accuracy & f1 accuracy
avg_test_accuracy = sum(test_accuracies) / len(test_accuracies)
print(f"Average Test Accuracy: {avg_test_accuracy * 100:.2f}%")
avg_f1_accuracy = sum(test_f1) / len(test_f1)
print(f"Average F1 Score: {avg_f1_accuracy * 100:.2f}%")

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch 1: Test Accuracy = 90.02%, F1 Score = 0.89
Epoch 2: Test Accuracy = 90.48%, F1 Score = 0.89
Epoch 3: Test Accuracy = 90.18%, F1 Score = 0.89
Average Test Accuracy: 90.23%
Average F1 Score: 89.12%


### multilevel

In [ ]:


# Custom Dataset class
class CustomMultilevelDataset(Dataset):
    def __init__(self, file_path):
        self.sentences, self.labels = self.load_data(file_path)

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        label = self.labels[index]

        return sentence, label

    def load_data(self, file_path):
        df = pd.read_csv(file_path)


        df = df[df['hate'] == 1].reset_index(drop=True)

        df['sentence'] = df.sentence.apply(preprocessing)
        df['sentence'] = df.sentence.apply(stopwordRemoval)
        df['sentence'] = df.sentence.apply(lambda x: Stemming(str(x)))


        df['category'] = df['category'].astype('category').cat.codes.astype('int64')

        df.info()
        sentences = df['sentence'].tolist()
        labels = df['category'].tolist()

        return sentences, labels


# Load the multilevel dataset
multilevel_dataset_file_path = "https://raw.githubusercontent.com/MU5HF1QUR/Thesis/main/Bengali%20hate%20speech%20.csv"
multilevel_dataset = CustomMultilevelDataset(multilevel_dataset_file_path)
sentences = multilevel_dataset.sentences
labels = multilevel_dataset.labels
print(len(multilevel_dataset), type(multilevel_dataset))
print(len(multilevel_dataset.sentences), type(multilevel_dataset.sentences))
print(len(multilevel_dataset.labels), type(multilevel_dataset.labels))


train_sentences, test_sentences, train_labels, test_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)


def tokenize_sentences(sentences, tokenizer):
    input_ids = []
    attention_masks = []

    for sentence in sentences:
        encoded_dict = tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=128,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=7)



train_input_ids, train_attention_masks = tokenize_sentences(train_sentences, tokenizer)


test_input_ids, test_attention_masks = tokenize_sentences(test_sentences, tokenizer)


batch_size = 16

train_data = TensorDataset(train_input_ids, train_attention_masks, torch.tensor(train_labels))
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)


test_data = TensorDataset(test_input_ids, test_attention_masks, torch.tensor(test_labels))
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

test_accuracies = []
test_f1 = []

for epoch in range(10):
    model.train()

    for batch in train_dataloader:
        batch_input_ids = batch[0].to(device)
        batch_attention_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)

        logits = outputs.logits

        loss = loss_fn(logits, batch_labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

    model.eval()

    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        all_predicted_labels = []
        all_true_labels = []

        for batch in test_dataloader:
            batch_input_ids = batch[0].to(device)
            batch_attention_masks = batch[1].to(device)
            batch_labels = batch[2].to(device)

            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)
            logits = outputs.logits

            _, predicted_labels = torch.max(logits, dim=1)
            total_correct += (predicted_labels == batch_labels).sum().item()
            total_samples += batch_labels.size(0)

            all_predicted_labels.extend(predicted_labels.cpu().numpy())
            all_true_labels.extend(batch_labels.cpu().numpy())

        accuracy = total_correct / total_samples
        test_accuracies.append(accuracy)
        f1 = f1_score(all_true_labels, all_predicted_labels, average='macro')
        test_f1.append(f1)

        print(f"Epoch {epoch+1}: Test Accuracy = {accuracy * 100:.2f}%, F1 Score = {f1:.2f}")

# Calculate the average test accuracy & f1 accuracy for multilevel classification
avg_test_accuracy = sum(test_accuracies) / len(test_accuracies)
print(f"Average Test Accuracy for Multilevel Classification: {avg_test_accuracy * 100:.2f}%")

avg_f1_accuracy = sum(test_f1) / len(test_f1)
print(f"Average F1 Score for Multilevel Classification: {avg_f1_accuracy * 100:.2f}%")


Streaming output truncated to the last 5000 lines.
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch 1: Test Accuracy = 52.95%, F1 Score = 0.52
Epoch 2: Test Accuracy = 60.50%, F1 Score = 0.61
Epoch 3: Test Accuracy = 59.65%, F1 Score = 0.60
Epoch 4: Test Accuracy = 61.70%, F1 Score = 0.63
Epoch 5: Test Accuracy = 63.55%, F1 Score = 0.64
Epoch 6: Test Accuracy = 63.40%, F1 Score = 0.63
Epoch 7: Test Accuracy = 61.85%, F1 Score = 0.63
Epoch 8: Test Accuracy = 61.60%, F1 Score = 0.63
Epoch 9: Test Accuracy = 62.40%, F1 Score = 0.63
Epoch 10: Test Accuracy = 60.20%, F1 Score = 0.61
Average Test Accuracy for Multilevel Classification: 60.78%
Average F1 Score for Multilevel Classification: 61.33%
